In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

In [2]:
url = "https://soundcloud.com/discover"

In [3]:
browser = webdriver.Chrome()
browser.get(url)

In [4]:
soup = BeautifulSoup(browser.page_source)

In [5]:
tracks = []
playlists = []
users = []

# Lưu link profile của producer
urls_user = []
# Lưu link của playlist
urls_playlist = []

In [6]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_down():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

In [7]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_up():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(document.body.scrollHeight, 0);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(document.body.scrollHeight,0);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

In [8]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, sep='\t')

In [9]:
# Đồng ý cookies
cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
cookies.click()

In [10]:
# bấm nút > đến hết
soup = BeautifulSoup(browser.page_source)
genres = soup.find_all("li", "mixedModularHome__item")

for i in range(len(genres)):
    lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
    j = 4
    while True:
        if j > len(lists):
            break
        browser.find_elements(By.XPATH,"//button[contains(.,'forward')]")[i].click()
        j += 4

In [11]:
# lấy hết tất cả playlists
soup = BeautifulSoup(browser.page_source)  
genres = soup.find_all("li", "mixedModularHome__item")

for i in range(len(genres)):
    lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
    for pl in lists:
        title = pl.find("a", "playableTile__heading").get_text().strip()
        link = pl.find("a", "playableTile__artworkLink")["href"]
        if title not in playlists:
            playlists.append(title)
            urls_playlist.append("https://soundcloud.com" + link)

**Lấy tất cả users và tracks của một playlist**

In [12]:
def get_all_user_track_playlist(): 
    scroll_down()
    
    soup = BeautifulSoup(browser.page_source)

    # Trích xuất user và track trong 1 playlist
    search = soup.find_all("li", "systemPlaylistTrackList__item")
    for producer in search:
        user = producer.find("a", "trackItem__username").get_text()
        url = producer.find("a", "trackItem__username").get('href')
        track = producer.find("a", "trackItem__trackTitle").get_text()
        if user not in users:
            users.append(user)
            urls_user.append(url)
        if track not in tracks:
            tracks.append(track)

**Lấy tất cả tracks của một user**

**Lấy tất cả playlists của một user**

In [ ]:
def get_all_playlists_user():
    soup = BeautifulSoup(browser.page_source)
    
    posts = soup.find_all("li", "soundList__item")
    
    # kiểm tra có playlist hay không, nếu có chạy dưới, nếu không bỏ qua
    if posts not is None:
        scroll_down()
        soup = BeautifulSoup(browser.page_source)
        
        for playlist in posts:
            title = playlist.find("a", "soundTitle__title")
            if title is None:
                break
            playlists.append(title.get_text().strip())

In [ ]:
def parse_playlist_per_genre():
    for url in urls_user:
        time.sleep(2)
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(2)
        get_all_playlists_user()
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

In [13]:
def parse_user_track_per_genre():
    for url in urls_playlist:
        time.sleep(2)
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(2)
        get_all_user_track_playlist()
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

In [14]:
parse_playlist_per_genre()

In [ ]:
parse_user_track_per_genre()

**Lưu data vào file tsv**

In [16]:
outputFile = "user.tsv"
list_to_tsv_csv(users, outputFile)
outputFile = "track.tsv"
list_to_tsv_csv(tracks, outputFile)
outputFile = "playlist.tsv"
list_to_tsv_csv(playlists, outputFile)